### Sanity check for sgss taxonomy assignment

In [1]:
# input file
setwd("~/bugbank_data")
sgss_f <- "./ukb_sgss_extract_20211115.csv"
sgss_tax_f <- "./bb_pathogen_taxonomy_13032023.tsv"

# load input file
sgss_tax <- read.csv(sgss_tax_f, sep = "\t")
sgss <- read.csv(sgss_f, header = T)
colnames(sgss) <- c(
    "SPECIMEN_NUMBER",
    "UKB_EID",
    "SPECIMEN_DATE",
    "LAB_REPORT_DATE",
    "REPORTING_LAB_NAME",
    "LAB_GEOG_NAME_CURRENT",
    "LOCAL_AUTHORITY_NAME",
    "SPECIMEN_GROUP_DESC",
    "SPECIMEN_TYPE_DESC",
    "ORGANISM_CATEGORY_DESC",
    "ORGANISM_GENUS_NAME",
    "ORGANISM_SPECIES_NAME",
    "ORGANISM_SUBSPECIES_NAME"
)

In [2]:
sgss_pathogens <- sgss$ORGANISM_SPECIES_NAME
setdiff(sgss_pathogens, sgss_tax$origin_name)
setdiff(sgss_tax$origin_name, sgss_pathogens)

character(0)

character(0)